In [1]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install sentencepiece

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-pwjz1pu4
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-pwjz1pu4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 870kB 9.0MB/s 
     |████████████████████████████████| 3.3MB 28.0MB/s 
  Created wheel for transformers: filename=transformers-4.6.0.dev0-cp37-none-any.whl size=2089853 sha256=ee7f4cbac9f798ac2bf13f07e3fb7c038091446e8067f196f5e2df07394c3290
  Stored in directory: /tmp/pip-ephem-wheel-cache-uljcf7d9/wheels/33/eb/3b/4bf5dd835e865e472d4fc0754f35ac0edb08fe852e8f21655f
Successfully built transformers
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=362f4b756ce27697776c4e1af4788b89ec8c1e9fd5c4e18c143372ceb686d35c
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366

In [33]:
from transformers import DistilBertTokenizerFast, LongformerTokenizerFast, BigBirdTokenizer
from transformers import TFDistilBertForSequenceClassification, TFLongformerForSequenceClassification, BigBirdForSequenceClassification

import tensorflow as tf
import json
import nltk
import re
import string

In [63]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)

In [64]:
# training_size = 20000

# with open("/content/complaints.json", 'r') as f:
#     datastore = json.load(f)

# sentences = []
# labels = []
# urls = []
# for item in datastore:
#   print(item)
#   sentences.append(item['headline'])
#   labels.append(item['is_sarcastic'])

# training_sentences = sentences[0:training_size]
# validation_sentences = sentences[training_size:]
# training_labels = labels[0:training_size]
# validation_labels = labels[training_size:]

In [65]:
# text preprocessing helper functions

def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.lower()
    # text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', ' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', ' ', text)
    text = re.sub('\w*\d\w*', ' ', text)
    return text


def text_preprocessing(text):
    """
    Cleaning and parsing the text.

    """
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    nopunc = clean_text(text)
    tokenized_text = tokenizer.tokenize(nopunc)
    #remove_stopwords = [w for w in tokenized_text if w not in stopwords.words('english')]
    combined_text = ' '.join(tokenized_text)
    return combined_text

In [67]:
nltk.download('words')
words = set(nltk.corpus.words.words())

def clean_sent(sent):
    return " ".join(w for w in nltk.wordpunct_tokenize(sent) \
     if w.lower() in words or not w.isalpha())

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [68]:
newsgroups_train.data = list(map(text_preprocessing, newsgroups_train.data))

In [69]:
newsgroups_train.data = list(map(clean_sent, newsgroups_train.data))

In [70]:
training_sentences, validation_sentences, training_labels, validation_labels = train_test_split(newsgroups_train.data, newsgroups_train.target, test_size=0.25, random_state=42)

In [71]:
training_sentences

['from subject re split organization brock university st tin version wrote concerning the split i personally am not in favor of doing this i learn an awful lot about all of graphics by reading this group from code to hardware to i just think making different out of this is a and will only result in a few a week per group i kind of like the convenience of one big forum for all of graphics anyone else feel this way just curious daemon what he said',
 'from subject where organization university of of computer science hi everybody can anyone name an anonymous where i can find the of the package portable map i would like to compile and run it on a sun thanks',
 'from subject re distribution organization ai center al i have before me a pertinent report from the united general accounting office national plane future research and development report number in the back it the following related key facing the program mar plane technology in japan and plane technology in technology technical data 

In [72]:
print(len(training_sentences))
print(len(validation_sentences))

1525
509


In [73]:
# tokenizer = LongformerTokenizerFast.from_pretrained('allenai/longformer-base-4096')

In [74]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [75]:
train_encodings = tokenizer(training_sentences,
                            truncation=True,
                            padding=True)
val_encodings = tokenizer(validation_sentences,
                            truncation=True,
                            padding=True)

In [76]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    training_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    validation_labels
))

In [77]:
# model = TFLongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096', num_labels=4)

In [78]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=4)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'activation_13', 'vocab_projector', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_119']
You should probably TRAIN this model on a down-stream task to be able to use 

In [79]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])
model.fit(train_dataset.shuffle(100).batch(16),
          epochs=4,
          batch_size=16,
          validation_data=val_dataset.shuffle(100).batch(16))

Epoch 1/4
96/96 [==============================] - 104s 1s/step - loss: 1.2535 - accuracy: 0.4907 - val_loss: 0.6252 - val_accuracy: 0.7407
Epoch 2/4
96/96 [==============================] - 96s 1s/step - loss: 0.5255 - accuracy: 0.8010 - val_loss: 0.4629 - val_accuracy: 0.8055
Epoch 3/4
96/96 [==============================] - 96s 1s/step - loss: 0.3371 - accuracy: 0.8711 - val_loss: 0.3708 - val_accuracy: 0.8527
Epoch 4/4
96/96 [==============================] - 96s 999ms/step - loss: 0.2242 - accuracy: 0.9339 - val_loss: 0.2970 - val_accuracy: 0.8919


In [80]:
model.save_pretrained("distilbert_model")
loaded_model = TFDistilBertForSequenceClassification.from_pretrained("distilbert_model")

Some layers from the model checkpoint at distilbert_model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_119']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert_model and are newly initialized: ['dropout_139']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [60]:
# model.save_pretrained("longformer_model")
# loaded_model = TFLongformerForSequenceClassification.from_pretrained("longformer_model")

In [61]:
test_sentence = "With their homes in ashes, residents share harrowing tales of survival after massive wildfires kill 15"
test_sentence_sarcasm = "News anchor hits back at viewer who sent her snarky note about ‘showing too much cleavage’ during broadcast"

# replace to test_sentence_sarcasm variable, if you want to test sarcasm
predict_input = tokenizer.encode(test_sentence,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")

tf_output = loaded_model.predict(predict_input)[0]
print(tf_output)

[[ 0.20116661 -0.5759988   0.01622463  0.31060493]]


In [62]:
tf_prediction = tf.nn.softmax(tf_output, axis=1).numpy()[0]
print(tf_prediction)

[0.2935555  0.1349496  0.2439894  0.32750556]
